# 📖 Chapter 01 — Data Exploration

## 🎯 Objectives

In this chapter, we will explore the Taiwan tourism dataset from the government open data platform.

We will analyze the data structure, check data quality, and determine which fields are suitable for RAG implementation.

## 📥 Step 01 — Load and Inspect Data
Load the scenic spot JSON file and convert it to a pandas DataFrame for easier analysis.

In [61]:
import pandas as pd
from src.config import RAW_DATA_DIR

In [12]:
import json

with open(RAW_DATA_DIR / "scenic_spot.json", "r", encoding="utf-8-sig") as f:
    data = json.load(f)

attraction = data["XML_Head"]["Infos"]["Info"]
df = pd.DataFrame(attraction)

In [14]:
df.shape

(5086, 35)

## 🔍 Step 02 — Data Quality Check
Check for missing values, duplicates, and data distribution across different cities.

In [19]:
df.isna().sum()

Id                   0
Name                 0
Zone               271
Toldescribe          0
Description         23
Tel                  0
Add                455
Zipcode              0
Region              75
Town                75
Travellinginfo      16
Opentime             0
Picture1             3
Picdescribe1        23
Picture2           271
Picdescribe2       271
Picture3           271
Picdescribe3       271
Map                216
Gov                 23
Px                   0
Py                   0
Orgclass           216
Class1               0
Class2             216
Class3             216
Level              216
Website            193
Parkinginfo        216
Parkinginfo_Px    5084
Parkinginfo_Py    5084
Ticketinfo         216
Remarks            216
Keyword            271
Changetime          23
dtype: int64

In [20]:
df.duplicated().sum()

np.int64(0)

In [17]:
df.columns

Index(['Id', 'Name', 'Zone', 'Toldescribe', 'Description', 'Tel', 'Add',
       'Zipcode', 'Region', 'Town', 'Travellinginfo', 'Opentime', 'Picture1',
       'Picdescribe1', 'Picture2', 'Picdescribe2', 'Picture3', 'Picdescribe3',
       'Map', 'Gov', 'Px', 'Py', 'Orgclass', 'Class1', 'Class2', 'Class3',
       'Level', 'Website', 'Parkinginfo', 'Parkinginfo_Px', 'Parkinginfo_Py',
       'Ticketinfo', 'Remarks', 'Keyword', 'Changetime'],
      dtype='object')

In [ ]:
region_rank = (
    df.groupby("Region")["Id"]
    .count()
    .reset_index()
    .rename(columns={"Id": "Count"})
    .sort_values("Count", ascending=False)
    .reset_index(drop=True)
)  # Drop the old index column

region_rank.index = range(1, len(region_rank) + 1)
region_rank

,Region,Count
1,新北市,577
2,臺北市,449
3,臺中市,367
4,彰化縣,344
5,澎湖縣,309
6,屏東縣,297
7,高雄市,294
8,南投縣,276
9,雲林縣,256
10,桃園市,249


## 📝 Step 03 — Text Field Analysis
Identify text fields suitable for semantic search and analyze their content-length distributions.

In [40]:
df["Description"].head()

0    巧克力共和國是一座以巧克力為主題的觀光工廠，建築設計、館內主題設計皆以巧克力為主題，這裡也提...
1    林口酒廠為北部地區產能最大、設備最現代化的清酒、紅麴葡萄酒生產工廠。廠內有產品推廣中心，開放...
2    這棟建築乃為邱良功祖厝，位於金城鎮浯江街中段，屋中尚存有方形銅鏡一面，另有雕龍聖旨石二塊，高...
3    陳詩吟洋樓為清朝末年(1903)金門高坑人陳詩吟前往新加坡、印尼經商致富，於民國21年(19...
4    金門朱子祠，立於清康熙二十六年（1687年），設於浯江書院內。乾隆三十五年（1770年），通...
Name: Description, dtype: object

In [42]:
df["Description"].str.len().describe()

count    5063.000000
mean      108.662651
std       120.373077
min         0.000000
25%         0.000000
50%        63.000000
75%       170.000000
max       400.000000
Name: Description, dtype: float64

## ✅ Step 04 — RAG Field Selection
Decide which fields to use for RAG and design the document format for vector database storage.

In [45]:
df[["Name", "Description", "Add", "Opentime", "Ticketinfo"]].head()

,Name,Description,Add,Opentime,Ticketinfo
0,宏亞食品巧克力觀光工廠,巧克力共和國是一座以巧克力為主題的觀光工廠，建築設計、館內主題設計皆以巧克力為主題，這裡也提...,桃園縣八德市建國路386號,,收費方式請電洽
1,臺灣菸酒(股)公司林口觀光酒廠,林口酒廠為北部地區產能最大、設備最現代化的清酒、紅麴葡萄酒生產工廠。廠內有產品推廣中心，開放...,桃園縣龜山鄉文化一路55號,,收費方式請電洽
2,邱良功古厝,這棟建築乃為邱良功祖厝，位於金城鎮浯江街中段，屋中尚存有方形銅鏡一面，另有雕龍聖旨石二塊，高...,金門縣893金城鎮浯江街27號,全天,免費
3,陳詩吟洋樓,陳詩吟洋樓為清朝末年(1903)金門高坑人陳詩吟前往新加坡、印尼經商致富，於民國21年(19...,金門縣893金城鎮珠浦東路44號,未開放,免費
4,朱子祠,金門朱子祠，立於清康熙二十六年（1687年），設於浯江書院內。乾隆三十五年（1770年），通...,金門縣893金城鎮珠埔北路36號,每日08:00-21:00,免費


In [ ]:
# Establish a sample document


def create_document(row):
    # Combine the text columns
    content_parts = [
        f"Name: {row['Name']}",
        f"Region: {row['Region']}",
    ]

    # if column exists, return it, otherwise, return "", and "" is false in if
    if str(row.get("Add", "")).strip():
        content_parts.append(f"Address: {row['Add']}")

    if str(row.get("Description", "")).strip():
        content_parts.append(f"Description: {row['Description']}")

    if str(row.get("Opentime", "")).strip():
        content_parts.append(f"Open_time: {row['Opentime']}")

    if str(row.get("Ticketinfo", "")).strip():  # ← 加上 str()
        content_parts.append(f"Ticket: {row['Ticketinfo']}")

    # Filter the empty
    content = "\n".join([part for part in content_parts if part])

    return {
        "id": row["Id"],
        "content": content,
        "metadata": {
            "name": row["Name"],
            "region": row["Region"],
            "category": row.get("Class1", "Unknown"),
        },
    }

In [64]:
sample = create_document(df.iloc[0])
print(sample["content"])

Name: 宏亞食品巧克力觀光工廠
Region: 桃園市
Address: 桃園縣八德市建國路386號
Description: 巧克力共和國是一座以巧克力為主題的觀光工廠，建築設計、館內主題設計皆以巧克力為主題，這裡也提供豐富的巧克力相關知識，亦可以DIY創作巧克力，為一寓教於樂、適合親子休閒娛樂的絕佳去處。
Ticket: 收費方式請電洽
